## Some basic stuff

In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd

In [2]:
def getSoup(link):
    sleep(0.1)
    r = requests.get(link)
    return BeautifulSoup(r.text,'lxml')

### Teams and their codes

#### Will be useful later on i think 

In [3]:
codes = getSoup('https://nhl.cz/sezona/tymy')

In [4]:
team_info_list = codes.findAll('a', {'class':'box-team__head'})
code_list = [x["href"] for x in team_info_list]
team_list = [h2.text for h2 in codes.findAll('h2')]
dict_team_codes = {team_list[i]: code_list[i] for i in range(len(code_list))}
dict_team_codes

{'Anaheim Ducks': '/klub/2004',
 'Arizona Coyotes': '/klub/2005',
 'Boston Bruins': '/klub/1987',
 'Buffalo Sabres': '/klub/1989',
 'Calgary Flames': '/klub/2006',
 'Carolina Hurricanes': '/klub/1990',
 'Colorado Avalanche': '/klub/2008',
 'Columbus Blue Jackets': '/klub/1991',
 'Dallas Stars': '/klub/2009',
 'Detroit Red Wings': '/klub/1992',
 'Edmonton Oilers': '/klub/2010',
 'Florida Panthers': '/klub/1993',
 'Chicago Blackhawks': '/klub/2007',
 'Los Angeles Kings': '/klub/2011',
 'Minnesota Wild': '/klub/2012',
 'Montreal Canadiens': '/klub/1994',
 'Nashville Predators': '/klub/2013',
 'New Jersey Devils': '/klub/1995',
 'New York Islanders': '/klub/1996',
 'New York Rangers': '/klub/1997',
 'Ottawa Senators': '/klub/1998',
 'Philadelphia Flyers': '/klub/1999',
 'Pittsburgh Penguins': '/klub/2000',
 'San Jose Sharks': '/klub/2014',
 'Seattle Kraken': '/klub/2730',
 'St. Louis Blues': '/klub/2015',
 'Tampa Bay Lightning': '/klub/2001',
 'Toronto Maple Leafs': '/klub/2002',
 'Vancouv

In [5]:
#sezony zatim rucne neprijde mi ze by to slo od nekud vycist
#2017 je 2017/2018 atd.
season_codes = {'2017': '6049', '2018': '6270', '2019': '6494', '2020': '6673','2021': '6733'}

## How it works: one team and one team so far

## Sorry za mess, nektery variables jsou zbytecny, uklidim to pozdeji

##### Needs to be made to be more automated later
#### Later used: TOI = Team of Interest

In [6]:
team_of_interest = "Boston Bruins"
code_of_interest = dict_team_codes[team_of_interest]
season_of_interest = '2021'
code_of_season = season_codes[season_of_interest]
soup = getSoup(f'https://nhl.cz{code_of_interest}/zapasy?matchList-filter-season={season_of_interest}&matchList-filter-competition={code_of_season}')
soup

<!DOCTYPE html>
<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]--><!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]--><!--[if IE 8]>         <html class="no-js lt-ie9"> <![endif]--><!--[if gt IE 8]><!--><html class="no-js"> <!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="NHL.cz | Oficiální web českého hokeje. Extraliga, 1. liga, 2. liga, mládež, reprezentace, mistrovství světa, NHL, Champions Hockey League, evropské ligy" name="description"/>
<meta content="BPA sport marketing a.s." name="Author"/>
<meta content="no-cache" http-equiv="cache-control"/>
<meta content="no-cache" http-equiv="pragma"/>
<meta content="-1" http-equiv="expires"/>
<meta content="document" name="resource-type"/>
<meta content="general" name="rating"/>
<meta content="global" name="distribution"/>
<meta content="width=device-width, initial-scale=1,maximum-scale=1, user-scalable=no" name="view

In [7]:
all_games = soup.findAll('tr')

### Goals in a list

In [8]:
goals = [int(td.text) for td in soup.findAll('td', {'class':'preview__score'})]
goals[:5]

[3, 1, 6, 3, 1]

### Teams in a list

In [9]:
teams = [span.text for span in soup.findAll('span', {'class':'preview__name--long'})]
teams[:5]

['Boston Bruins',
 'Dallas Stars',
 'Philadelphia Flyers',
 'Boston Bruins',
 'Buffalo Sabres']

### Overtime or not

In [10]:
OT = []
for game in all_games:
    ot = game.find('span', {'class':'dot dot__blue'})
    if ot is None:
        OT.append(0)
    else:
        OT.append(1)
print(OT[:7])

[0, 0, 0, 0, 0, 0, 1]


### Game ID in a list

In [11]:
game_ids_list = soup.find_all("a", href=lambda value: value and value.startswith("/zapas/"))
game_ids = [x["href"] for x in game_ids_list][::2]
game_ids[:5]

['/zapas/2897006',
 '/zapas/2897032',
 '/zapas/2897045',
 '/zapas/2897063',
 '/zapas/2897081']

### Putting it in a dataframe

In [12]:
aux_dictionary = {'game_id':game_ids, 'team_home': teams[::2], 'team_away': teams[1::2],
    'goals_home': goals[::2], 'goals_away': goals[1::2], 'OT': OT} 
df = pd.DataFrame(aux_dictionary)
df.head()

,game_id,team_home,team_away,goals_home,goals_away,OT
0,/zapas/2897006,Boston Bruins,Dallas Stars,3,1,0
1,/zapas/2897032,Philadelphia Flyers,Boston Bruins,6,3,0
2,/zapas/2897045,Buffalo Sabres,Boston Bruins,1,4,0
3,/zapas/2897063,Boston Bruins,San Jose Sharks,4,3,0
4,/zapas/2897081,Florida Panthers,Boston Bruins,4,1,0


### Adding some more variables that might be useful

In [13]:
df.loc[(df["goals_home"] > df["goals_away"]) & (df["OT"] == 0), 'points_home'] = 3
df.loc[(df["goals_home"] > df["goals_away"]) & (df["OT"] == 1), 'points_home'] = 2
df.loc[(df["goals_home"] < df["goals_away"]) & (df["OT"] == 1), 'points_home'] = 1
df.loc[(df["goals_home"] < df["goals_away"]) & (df["OT"] == 0), 'points_home'] = 0

df.loc[(df["goals_home"] > df["goals_away"]) & (df["OT"] == 0), 'points_away'] = 0
df.loc[(df["goals_home"] > df["goals_away"]) & (df["OT"] == 1), 'points_away'] = 1
df.loc[(df["goals_home"] < df["goals_away"]) & (df["OT"] == 1), 'points_away'] = 2
df.loc[(df["goals_home"] < df["goals_away"]) & (df["OT"] == 0), 'points_away'] = 3

df.head()

,game_id,team_home,team_away,goals_home,goals_away,OT,points_home,points_away
0,/zapas/2897006,Boston Bruins,Dallas Stars,3,1,0,3.0,0.0
1,/zapas/2897032,Philadelphia Flyers,Boston Bruins,6,3,0,3.0,0.0
2,/zapas/2897045,Buffalo Sabres,Boston Bruins,1,4,0,0.0,3.0
3,/zapas/2897063,Boston Bruins,San Jose Sharks,4,3,0,3.0,0.0
4,/zapas/2897081,Florida Panthers,Boston Bruins,4,1,0,3.0,0.0


### Adding variable whether the team of interest is a home team

In [14]:
df.loc[df["team_home"] == team_of_interest, 'TOI_home'] = 1
df.loc[df["team_home"] != team_of_interest, 'TOI_home'] = 0

### Adding how many points TOI received

In [15]:
df.loc[(df["TOI_home"] == 1), 'TOI_points'] = df["points_home"]
df.loc[(df["TOI_home"] == 0), 'TOI_points'] = df["points_away"]

df.loc[(df["TOI_home"] == 1), 'TOI_goals_scored'] = df["goals_home"]
df.loc[(df["TOI_home"] == 0), 'TOI_goals_scored'] = df["goals_away"]
df.loc[(df["TOI_home"] == 1), 'TOI_goals_rec'] = df["goals_away"]
df.loc[(df["TOI_home"] == 0), 'TOI_goals_rec'] = df["goals_home"]

### Adding Output Variable: the result of our TOI

In [16]:
df.loc[df["OT"] == 1, 'TOI_result'] = "OT"
df.loc[(df["TOI_home"] == 1) & (df["points_home"] == 3), 'TOI_result'] = "W"
df.loc[(df["TOI_home"] == 1) & (df["points_home"] == 0), 'TOI_result'] = "L"
df.loc[(df["TOI_home"] == 0) & (df["points_home"] == 3), 'TOI_result'] = "L"
df.loc[(df["TOI_home"] == 0) & (df["points_home"] == 0), 'TOI_result'] = "W"

In [17]:
df.loc[(df["TOI_home"] == 1), 'other_team'] = df["team_away"]
df.loc[(df["TOI_home"] == 0), 'other_team'] = df["team_home"]
df.head()

,game_id,team_home,team_away,goals_home,goals_away,OT,points_home,points_away,TOI_home,TOI_points,TOI_goals_scored,TOI_goals_rec,TOI_result,other_team
0,/zapas/2897006,Boston Bruins,Dallas Stars,3,1,0,3.0,0.0,1.0,3.0,3.0,1.0,W,Dallas Stars
1,/zapas/2897032,Philadelphia Flyers,Boston Bruins,6,3,0,3.0,0.0,0.0,0.0,3.0,6.0,L,Philadelphia Flyers
2,/zapas/2897045,Buffalo Sabres,Boston Bruins,1,4,0,0.0,3.0,0.0,3.0,4.0,1.0,W,Buffalo Sabres
3,/zapas/2897063,Boston Bruins,San Jose Sharks,4,3,0,3.0,0.0,1.0,3.0,4.0,3.0,W,San Jose Sharks
4,/zapas/2897081,Florida Panthers,Boston Bruins,4,1,0,3.0,0.0,0.0,0.0,1.0,4.0,L,Florida Panthers


### Adding variables - sum of last 20 games

In [18]:
TOI_points_tot = [None] * 20
TOI_goals_scored_tot = [None] * 20
TOI_goals_rec_tot = [None] * 20
for x in range(len(df)):
    if x > 19:
        TOI_points_tot.append(int(df[['TOI_points']][x-20:x].sum()))
        TOI_goals_scored_tot.append(int(df[['TOI_goals_scored']][x-20:x].sum()))
        TOI_goals_rec_tot.append(int(df[['TOI_goals_rec']][x-20:x].sum()))
df['TOI_points_tot'] = TOI_points_tot
df['TOI_goals_scored_tot'] = TOI_goals_scored_tot
df['TOI_goals_rec_tot'] = TOI_goals_rec_tot

df.iloc[19:25, 8:] 

,TOI_home,TOI_points,TOI_goals_scored,TOI_goals_rec,TOI_result,other_team,TOI_points_tot,TOI_goals_scored_tot,TOI_goals_rec_tot
19,0.0,3.0,2.0,0.0,W,Nashville Predators,NaN,NaN,NaN
20,1.0,1.0,2.0,3.0,OT,Tampa Bay Lightning,35.0,59.0,53.0
21,0.0,1.0,1.0,2.0,OT,Vancouver Canucks,33.0,58.0,55.0
22,0.0,3.0,3.0,2.0,W,Edmonton Oilers,34.0,56.0,51.0
23,0.0,3.0,4.0,2.0,W,Calgary Flames,34.0,55.0,52.0
24,1.0,0.0,1.0,4.0,L,Vegas Golden Knights,34.0,55.0,51.0


In [19]:
#tady jen neco zkousim
df[['TOI_home']][:20]
int(df[['TOI_points']][0:20].sum())

35

Takhle bude vypadat df, akorat v nim bude dat za vice sezon

## Creating the dataset later used for modellig

## ke kazdemu zapasu: 
- jeho id
- nazev druheho tymu
- vysledek TOI (output variable)
- jestli hraje TOI doma
- kolik ziskal TOI bodu v poslednich X (20/30) zapasech
- kolik dal TOI golu v poslednich X (20/30) zapasech
- kolik dostal TOI golu v poslednich X (20/30) zapasech

  
    - toto bude jeste vse potreba i pro druhy tym s kterym je prave ten zapas
    - tyhle radky plus radky o druhem tymu by uz meli jit nakrmit do modelu a to by melo byt vsechno

In [20]:
n_of_games = 20 #the number of previous games we will take into account, myslim ze 20 je tak optimalni

In [21]:
df_model = df[['game_id','TOI_result','other_team','TOI_home', 'TOI_points_tot', 'TOI_goals_scored_tot', 'TOI_goals_rec_tot']][20:].copy()
df_model

,game_id,TOI_result,other_team,TOI_home,TOI_points_tot,TOI_goals_scored_tot,TOI_goals_rec_tot
20,/zapas/2897351,OT,Tampa Bay Lightning,1.0,35.0,59.0,53.0
21,/zapas/2897379,OT,Vancouver Canucks,0.0,33.0,58.0,55.0
22,/zapas/2897387,W,Edmonton Oilers,0.0,34.0,56.0,51.0
23,/zapas/2897403,W,Calgary Flames,0.0,34.0,55.0,52.0
24,/zapas/2897426,L,Vegas Golden Knights,1.0,34.0,55.0,51.0
...,...,...,...,...,...,...,...
77,/zapas/2898234,W,New York Rangers,1.0,34.0,63.0,54.0
78,/zapas/2897458,W,Montreal Canadiens,0.0,34.0,62.0,52.0
79,/zapas/2898259,W,Florida Panthers,1.0,34.0,64.0,53.0
80,/zapas/2898274,W,Buffalo Sabres,1.0,35.0,66.0,54.0


# Turning it into OOP

In [22]:
def create_link(team_of_interest,season_of_interest):
    code_of_season = season_codes[season_of_interest]
    code_of_interest = dict_team_codes[team_of_interest]
    return f'https://nhl.cz{code_of_interest}/zapasy?matchList-filter-season={season_of_interest}&matchList-filter-competition={code_of_season}'
create_link('Boston Bruins', '2021')

'https://nhl.cz/klub/1987/zapasy?matchList-filter-season=2021&matchList-filter-competition=6733'

In [23]:
#tady bude Class co idealne vrati df ktery pak pouzijem, muzem z toho vytisknout i nakej v prubehu tak jako KUK
class One_team_season:
    def __init__(self,link,team):
        self.link = link
        self.team = team
    
    def get_soup(self):
        r = requests.get(self.link)
        if r.status_code == 200:
            r.encoding='UTF-8'
            return BeautifulSoup(r.text)
        else:
            raise Exception(f'Cannot access the url. Returning status_code {r.status_code}')
            
    def get_goals(self,soup):
        return [int(td.text) for td in soup.findAll('td', {'class':'preview__score'})]
    
    def get_teams(self,soup):
        return [span.text for span in soup.findAll('span', {'class':'preview__name--long'})]
    
    def get_OT(self,soup):
        all_games = soup.findAll('tr')
        OT = []
        for game in all_games:
            ot = game.find('span', {'class':'dot dot__blue'})
            if ot is None:
                OT.append(0)
            else:
                OT.append(1)
        return OT
    
    def get_game_IDs(self,soup):
        game_ids_list = soup.find_all("a", href=lambda value: value and value.startswith("/zapas/"))
        game_ids = [x["href"] for x in game_ids_list][::2]
        return game_ids
    
    def create_df(self):
        soup = self.get_soup()
        goals = self.get_goals(soup)
        teams = self.get_teams(soup)
        OT = self.get_OT(soup)
        game_ids = self.get_game_IDs(soup)
        aux_dictionary = {'game_id':game_ids, 'team_home': teams[::2], 'team_away': teams[1::2],
        'goals_home': goals[::2], 'goals_away': goals[1::2], 'OT': OT} 
        df = pd.DataFrame(aux_dictionary)
        return df
    
    def create_model_df(self):
        df = self.create_df()
        df.loc[(df["goals_home"] > df["goals_away"]) & (df["OT"] == 0), 'points_home'] = 3
        df.loc[(df["goals_home"] > df["goals_away"]) & (df["OT"] == 1), 'points_home'] = 2
        df.loc[(df["goals_home"] < df["goals_away"]) & (df["OT"] == 1), 'points_home'] = 1
        df.loc[(df["goals_home"] < df["goals_away"]) & (df["OT"] == 0), 'points_home'] = 0

        df.loc[(df["goals_home"] > df["goals_away"]) & (df["OT"] == 0), 'points_away'] = 0
        df.loc[(df["goals_home"] > df["goals_away"]) & (df["OT"] == 1), 'points_away'] = 1
        df.loc[(df["goals_home"] < df["goals_away"]) & (df["OT"] == 1), 'points_away'] = 2
        df.loc[(df["goals_home"] < df["goals_away"]) & (df["OT"] == 0), 'points_away'] = 3
        
        df.loc[df["team_home"] == self.team, 'TOI_home'] = 1
        df.loc[df["team_home"] != self.team, 'TOI_home'] = 0
        
        df.loc[(df["TOI_home"] == 1), 'TOI_points'] = df["points_home"]
        df.loc[(df["TOI_home"] == 0), 'TOI_points'] = df["points_away"]

        df.loc[(df["TOI_home"] == 1), 'TOI_goals_scored'] = df["goals_home"]
        df.loc[(df["TOI_home"] == 0), 'TOI_goals_scored'] = df["goals_away"]
        df.loc[(df["TOI_home"] == 1), 'TOI_goals_rec'] = df["goals_away"]
        df.loc[(df["TOI_home"] == 0), 'TOI_goals_rec'] = df["goals_home"]
        
        df.loc[df["OT"] == 1, 'TOI_result'] = "OT"
        df.loc[(df["TOI_home"] == 1) & (df["points_home"] == 3), 'TOI_result'] = "W"
        df.loc[(df["TOI_home"] == 1) & (df["points_home"] == 0), 'TOI_result'] = "L"
        df.loc[(df["TOI_home"] == 0) & (df["points_home"] == 3), 'TOI_result'] = "L"
        df.loc[(df["TOI_home"] == 0) & (df["points_home"] == 0), 'TOI_result'] = "W"
        
        df.loc[(df["TOI_home"] == 1), 'other_team'] = df["team_away"]
        df.loc[(df["TOI_home"] == 0), 'other_team'] = df["team_home"]
        
        return df


    

In [24]:
boston = One_team_season('https://nhl.cz/klub/1987/zapasy?matchList-filter-season=2021&matchList-filter-competition=6733',
                        'Boston Bruins')
boston.create_df()

,game_id,team_home,team_away,goals_home,goals_away,OT
0,/zapas/2897006,Boston Bruins,Dallas Stars,3,1,0
1,/zapas/2897032,Philadelphia Flyers,Boston Bruins,6,3,0
2,/zapas/2897045,Buffalo Sabres,Boston Bruins,1,4,0
3,/zapas/2897063,Boston Bruins,San Jose Sharks,4,3,0
4,/zapas/2897081,Florida Panthers,Boston Bruins,4,1,0
...,...,...,...,...,...,...
77,/zapas/2898234,Boston Bruins,New York Rangers,3,1,0
78,/zapas/2897458,Montreal Canadiens,Boston Bruins,3,5,0
79,/zapas/2898259,Boston Bruins,Florida Panthers,4,2,0
80,/zapas/2898274,Boston Bruins,Buffalo Sabres,5,0,0


In [25]:
boston.create_model_df()

,game_id,team_home,team_away,goals_home,goals_away,OT,points_home,points_away,TOI_home,TOI_points,TOI_goals_scored,TOI_goals_rec,TOI_result,other_team
0,/zapas/2897006,Boston Bruins,Dallas Stars,3,1,0,3.0,0.0,1.0,3.0,3.0,1.0,W,Dallas Stars
1,/zapas/2897032,Philadelphia Flyers,Boston Bruins,6,3,0,3.0,0.0,0.0,0.0,3.0,6.0,L,Philadelphia Flyers
2,/zapas/2897045,Buffalo Sabres,Boston Bruins,1,4,0,0.0,3.0,0.0,3.0,4.0,1.0,W,Buffalo Sabres
3,/zapas/2897063,Boston Bruins,San Jose Sharks,4,3,0,3.0,0.0,1.0,3.0,4.0,3.0,W,San Jose Sharks
4,/zapas/2897081,Florida Panthers,Boston Bruins,4,1,0,3.0,0.0,0.0,0.0,1.0,4.0,L,Florida Panthers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,/zapas/2898234,Boston Bruins,New York Rangers,3,1,0,3.0,0.0,1.0,3.0,3.0,1.0,W,New York Rangers
78,/zapas/2897458,Montreal Canadiens,Boston Bruins,3,5,0,0.0,3.0,0.0,3.0,5.0,3.0,W,Montreal Canadiens
79,/zapas/2898259,Boston Bruins,Florida Panthers,4,2,0,3.0,0.0,1.0,3.0,4.0,2.0,W,Florida Panthers
80,/zapas/2898274,Boston Bruins,Buffalo Sabres,5,0,0,3.0,0.0,1.0,3.0,5.0,0.0,W,Buffalo Sabres


In [ ]:
#dale je potreba: mergnout datasety pro X poslednich sezon dohromady pro TOI
#a creatnout ten df s info ze poslednich 20 zapasu
#asi bude nejlepsi kdyz se takovej dataset udela pro vsech 32 tymu protoze ty data pak budou potreba na training
#team of interest nak poresit
#hodit treba i ten create link nejak do toho Classu, aby pak stacil:
#"boston = One_team_season('Boston Bruins', 2021)"... a ono to vrati ten DF


### Calculating Home Advantage

In [ ]:
sum_wins_home = sum(df["goals_home"] > df["goals_away"])
sum_wins_away = sum(df["goals_home"] < df["goals_away"])
 
home_winrate = sum_wins_home/(sum_wins_home+sum_wins_away)
home_winrate

### List of all the teams

In [ ]:
distinct_teams = []
for team in teams:
    if team not in distinct_teams:
        distinct_teams.append(team)
print(distinct_teams)

In [ ]:
df

### tohle moc nechapu k cemu je

In [ ]:
points_home = df.groupby(['team_home'])['points_home'].sum().reset_index()
points_away = df.groupby(['team_away'])['points_away'].sum().reset_index()
tp = points_home.merge(points_away,left_on = 'team_home',right_on = 'team_away')
teamm = tp["team_home"]
tp["Team"] = teamm
points_total = tp["points_home"] + tp["points_away"]
tp["points_total"] = points_total
tp = tp.drop('points_home', 1)
tp = tp.drop('points_away', 1)
tp = tp.drop('team_away', 1)
tp = tp.drop('team_home', 1)
tp